In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/08 13:51:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pyspark.__version__

'3.3.2'

In [4]:
df = spark.read \
    .option("header", "true") \
    .parquet("/home/dpeter/data-engineering-zoomcamp/05-batch/code/module_05_homework/yellow_tripdata_2024-10.parquet")

In [5]:
output_path = "/home/dpeter/data-engineering-zoomcamp/05-batch/code/module_05_homework/output"

df \
    .repartition(4) \
    .write \
    .mode("overwrite") \
    .parquet(output_path)

In [6]:
df.registerTempTable('df_sql')

/home/dpeter/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [7]:
df_table = spark.sql(""" 
                    SELECT * FROM df_sql""")
df_table.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [8]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [10]:
df_question_3 = spark.sql("""
    SELECT COUNT(*)
    FROM df_sql
    WHERE date_format(tpep_pickup_datetime, 'yyyy-MM-dd') = '2024-10-15'
""") 
df_question_3.show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [10]:
df_question_4 = spark.sql("""
    SELECT 
           (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS trip_duration_hours
    FROM 
        df_sql
    ORDER BY 
        trip_duration_hours DESC
    LIMIT 1
""")


df_question_4.show()

+-------------------+
|trip_duration_hours|
+-------------------+
| 162.61777777777777|
+-------------------+



In [17]:
csv_path = "/home/dpeter/data-engineering-zoomcamp/05-batch/code/module_05_homework/taxi_zone_lookup.csv"
taxi_zone_df = spark.read.csv(csv_path, header=True, inferSchema=True)
taxi_zone_df.createOrReplaceTempView("taxi_zone_lookup")
taxi_zone_df.show(100)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [18]:
df_question_6 = spark.sql("""
    SELECT 
           PULocationID AS pickup_location_id,
            Zone AS pickup_location,
           COUNT(*)
    FROM 
        df_sql
    INNER JOIN
        taxi_zone_lookup
    ON
        df_sql.PULocationID = taxi_zone_lookup.LocationID
    GROUP BY
        1,2
    ORDER BY
        3 
""")
df_question_6.show()

+------------------+--------------------+--------+
|pickup_location_id|     pickup_location|count(1)|
+------------------+--------------------+--------+
|               105|Governor's Island...|       1|
|               199|       Rikers Island|       2|
|                 5|       Arden Heights|       2|
|                 2|         Jamaica Bay|       3|
|               111| Green-Wood Cemetery|       3|
|               204|   Rossville/Woodrow|       4|
|               187|       Port Richmond|       4|
|               245|       West Brighton|       4|
|                44|Charleston/Totten...|       4|
|                84|Eltingville/Annad...|       4|
|               109|         Great Kills|       6|
|                59|        Crotona Park|       6|
|               156|     Mariners Harbor|       7|
|               118|Heartland Village...|       7|
|               176|             Oakwood|       9|
|               206|Saint George/New ...|       9|
|                30|       Broa